<a href="https://colab.research.google.com/github/man0045/Data-Science/blob/master/Practice_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] ="jax"

import keras

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [3]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         147584    
                                                                 
 global_average_pooling2d (  (None, 128)               0         
 GlobalAveragePooling2D)                                

In [5]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [ ]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
]
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs= epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose = 0)

Epoch 1/20
399/399 [==============================] - 341s 849ms/step - loss: 0.8107 - acc: 0.7199 - val_loss: 0.1641 - val_acc: 0.9550
Epoch 2/20
399/399 [==============================] - 325s 816ms/step - loss: 0.2350 - acc: 0.9313 - val_loss: 0.1217 - val_acc: 0.9628
Epoch 3/20
399/399 [==============================] - 328s 823ms/step - loss: 0.1620 - acc: 0.9526 - val_loss: 0.0817 - val_acc: 0.9754
Epoch 4/20
399/399 [==============================] - 319s 799ms/step - loss: 0.1289 - acc: 0.9620 - val_loss: 0.0667 - val_acc: 0.9802
Epoch 5/20
399/399 [==============================] - 322s 808ms/step - loss: 0.1126 - acc: 0.9671 - val_loss: 0.0470 - val_acc: 0.9860
Epoch 6/20
399/399 [==============================] - 325s 814ms/step - loss: 0.0973 - acc: 0.9713 - val_loss: 0.0390 - val_acc: 0.9888
Epoch 7/20
399/399 [==============================] - 325s 815ms/step - loss: 0.0871 - acc: 0.9748 - val_loss: 0.0357 - val_acc: 0.9900
Epoch 8/20
399/399 [============================

In [ ]:
model.save("final_model.keras")

In [ ]:
model=keras.saving.load_model("final_model.keras")

In [ ]:
predictions=model.predict(x_test)